In [1]:
dogersData = "../DataSet/Dodgers.data"
dogersEvent = "../DataSet/Dodgers.events"
dogersDRDD = sc.textFile(dogersData)
dogersERDD = sc.textFile(dogersEvent)

In [2]:
from datetime import datetime

In [3]:
# we can also use csv and StringIO module which we see in Transforming and Cleaning Unstructured Data jupyter notebook refer it.
def parseDogersDataRdd(row):
    reader = row.split(",")
    reader[0] = datetime.strptime(reader[0], "%m/%d/%Y %H:%M")
    reader[1] = int(reader[1])
    return tuple(reader)

In [4]:
dogersDRDD2 = dogersDRDD.map(parseDogersDataRdd)
dogersDRDD2.take(5)

[(datetime.datetime(2005, 4, 10, 0, 0), -1),
 (datetime.datetime(2005, 4, 10, 0, 5), -1),
 (datetime.datetime(2005, 4, 10, 0, 10), -1),
 (datetime.datetime(2005, 4, 10, 0, 15), -1),
 (datetime.datetime(2005, 4, 10, 0, 20), -1)]

In [25]:
#summarizing a Pair RDD by reducebykey method
newDogersDRDD = dogersDRDD2.map(lambda x: (x[0].date(), x[1]))\
                            .reduceByKey(lambda x, y: x + y)

In [26]:
# For Ascending order just mention the column which u want to sort in lambda function
# so lambda Function consider it as such
newDogersDRDD.sortBy(lambda x: x[1]).take(10)

[(datetime.date(2005, 4, 10), -288),
 (datetime.date(2005, 10, 1), -260),
 (datetime.date(2005, 6, 28), -96),
 (datetime.date(2005, 7, 4), 328),
 (datetime.date(2005, 7, 12), 1204),
 (datetime.date(2005, 5, 23), 2173),
 (datetime.date(2005, 9, 17), 2426),
 (datetime.date(2005, 9, 10), 2851),
 (datetime.date(2005, 6, 27), 2907),
 (datetime.date(2005, 7, 10), 3518)]

In [27]:
# For Ascending order just mention the column with negative sign in lambda function which u want to sort
# here the +ve number becomes -ve and all -ve number becomes +ve.
newDogersDRDD.sortBy(lambda x: -x[1]).take(10)

[(datetime.date(2005, 7, 28), 7661),
 (datetime.date(2005, 7, 29), 7499),
 (datetime.date(2005, 8, 12), 7287),
 (datetime.date(2005, 7, 27), 7238),
 (datetime.date(2005, 9, 23), 7175),
 (datetime.date(2005, 7, 26), 7163),
 (datetime.date(2005, 5, 20), 7119),
 (datetime.date(2005, 8, 11), 7110),
 (datetime.date(2005, 9, 8), 7107),
 (datetime.date(2005, 9, 7), 7082)]

In [28]:
dogersERDD.first()

'04/12/05,13:10:00,16:23:00,55892,San Francisco,W 9-8�'

In [29]:
def parseDogersEventRDD(row):
    reader = row.split(",")
    reader[0] = datetime.strptime(reader[0], "%m/%d/%y").date()
    reader[4] = reader[4]
    return (reader[0], reader[4])

In [30]:
parseDogersEventRDD(dogersERDD.first())

(datetime.date(2005, 4, 12), 'San Francisco')

In [31]:
newDogersERDD2 = dogersERDD.map(parseDogersEventRDD)

In [32]:
newDogersERDD2.first()

(datetime.date(2005, 4, 12), 'San Francisco')

In [33]:
newDogers = newDogersDRDD.leftOuterJoin(newDogersERDD2)

In [15]:
newDogers.take(10)

[(datetime.date(2005, 4, 11), (5062, None)),
 (datetime.date(2005, 4, 15), (6459, 'San Diego')),
 (datetime.date(2005, 4, 17), (5322, 'San Diego')),
 (datetime.date(2005, 4, 19), (6049, None)),
 (datetime.date(2005, 4, 21), (5977, None)),
 (datetime.date(2005, 4, 22), (6038, None)),
 (datetime.date(2005, 4, 23), (5366, None)),
 (datetime.date(2005, 4, 24), (4319, None)),
 (datetime.date(2005, 4, 25), (6280, 'Arizona')),
 (datetime.date(2005, 4, 30), (6090, 'Colorado'))]

In [34]:
def regOrGame(row):
    if row[1][1] == None:
        return (row[0], row[1][1], "Regular Day", row[1][0])
    else:
        return (row[0], row[1][1], "Game Day", row[1][0])

newDogers2 = newDogers.map(regOrGame)

In [36]:
newDogers2.take(5)

[(datetime.date(2005, 4, 11), None, 'Regular Day', 5062),
 (datetime.date(2005, 4, 15), 'San Diego', 'Game Day', 6459),
 (datetime.date(2005, 4, 17), 'San Diego', 'Game Day', 5322),
 (datetime.date(2005, 4, 19), None, 'Regular Day', 6049),
 (datetime.date(2005, 4, 21), None, 'Regular Day', 5977)]

In [39]:
newDogers2 = newDogers2.sortBy(lambda x: -x[3])

In [46]:
newDogers2.take(10)

[(datetime.date(2005, 7, 28), 'Cincinnati', 'Game Day', 7661),
 (datetime.date(2005, 7, 29), 'St. Louis', 'Game Day', 7499),
 (datetime.date(2005, 8, 12), 'NY Mets', 'Game Day', 7287),
 (datetime.date(2005, 7, 27), 'Cincinnati', 'Game Day', 7238),
 (datetime.date(2005, 9, 23), 'Pittsburgh', 'Game Day', 7175),
 (datetime.date(2005, 7, 26), 'Cincinnati', 'Game Day', 7163),
 (datetime.date(2005, 5, 20), 'LA Angels', 'Game Day', 7119),
 (datetime.date(2005, 8, 11), 'Philadelphia', 'Game Day', 7110),
 (datetime.date(2005, 9, 8), None, 'Regular Day', 7107),
 (datetime.date(2005, 9, 7), 'San Francisco', 'Game Day', 7082)]

# CombineByKey

In [53]:
newDogers2.map(lambda x: (x[2], x[3]))\
            .combineByKey(lambda val: (val, 1), \
                         lambda x, val: (x[0] + val, x[1] + 1), \
                         lambda x1, x2: (x1[0] + x2[0], x1[1] + x2[1]))\
            .mapValues(lambda x: x[0] / x[1]).collect()

[('Regular Day', 5411.329787234043), ('Game Day', 5948.604938271605)]